In [ ]:
from sklearn.cluster import KMeans
from scipy.special import logsumexp
import numpy as np
import matplotlib.pyplot as plt
import math
import os

## Train HMM

In [ ]:
def trainKMeans(gestureType, data_dir, M):    
    data = np.array([])
    print("Loading training data...")
    for filename in os.listdir(data_dir):
        if filename.endswith(".txt"):
            if gestureType not in filename:
                continue
            temp = np.loadtxt(data_dir + filename)
    #         print(temp.shape)
            if len(data) == 0:
                data = temp[:,1:]
            else:
                data = np.concatenate((data, temp[:,1:]))
    #         print(data.shape)
    #         print(data)
            kmeans = KMeans(n_clusters=M).fit(data)
    print("Done!")

    plt.figure(figsize=(16, 12))
    for i in range(data.shape[1] - 1):
        plt.scatter(data[:,i], data[:,i+1], c=kmeans.labels_.astype(float), s=0.4)
    for i in range(data.shape[1] - 1):
        plt.scatter(kmeans.cluster_centers_[:,i], kmeans.cluster_centers_[:,i+1], c='r', s=10)
    return kmeans

In [ ]:
# fig = plt.figure(figsize=(16, 12))
# ax = fig.add_subplot(projection='3d')
# for i in range(data.shape[1] - 2):
#     plt.scatter(data[:,i], data[:,i+1], data[:,i+2], c=kmeans.labels_.astype(float))
# for i in range(data.shape[1] - 2):
#     plt.scatter(kmeans.cluster_centers_[:,i], kmeans.cluster_centers_[:,i+1], kmeans.cluster_centers_[:,i+2], c='r')
# # plt.scatter(data[:,0], data[:,1])

In [ ]:
def forward(O, A, B, pi):
    # alpha[t][i] where 1 < t < len(O), 1 < i < len(A)
    alpha = np.zeros([len(O), len(A)])
    alphaDots = np.zeros([len(O), len(A)])
    alphaHat = np.zeros([len(O), len(A)])
    c = np.zeros(len(O))
    alpha[0, :] = pi * B[:, int(O[0])]
    alphaDots[0] = alpha[0]
    c[0] = 1 / np.sum(alphaDots[0])
    alphaHat[0] = c[0] * alphaDots[0]
    for t in range(1, len(O)):
        for i in range(len(A)):
#             No scaling
#             alpha[t, i] = alpha[t - 1].dot(A[i] * B[i, int(O[t])])
#             Scaling
            alphaDots[t, i] = (alphaHat[t - 1]).dot(A[i] * B[i, int(O[t])])
        c[t] = 1 / np.sum(alphaDots[t])
        alphaHat[t] = c[t] * alphaDots[t]
    
    return alphaHat, c

In [ ]:
def backward(O, A, B, c):
    beta = np.zeros([len(O), len(A)])
    betaHat = np.zeros([len(O), len(A)])
    beta[len(O) - 1] = np.ones(len(A))
    betaHat[-1] = c[-1] * beta[-1]
    for t in reversed(range(len(O) - 1)):
        for i in range(len(A)):
#             No scaling
#             beta[t][i] = A[:, i].dot(B[:, int(O[t + 1])] * beta[t + 1])
#             Scaling
            beta[t][i] = A[:, i].dot(B[:, int(O[t + 1])] * betaHat[t + 1])
        betaHat[t] = c[t] * beta[t]
    
    return betaHat

In [ ]:
def EStep(O, A, B, alpha, beta, c):
    gamma = np.zeros([len(O), len(A)])
    for t in range(len(O)):
        denominator = alpha[t].dot(beta[t])
        for i in range(len(A)):
            numerator = alpha[t, i] * beta[t, i]
            gamma[t,i] = numerator / denominator
    
    zeta = np.zeros([len(O), len(A), len(A)])
    for t in range(len(O) - 1):
        denominator = 0
        for i in range(len(A)):
            for j in range(len(A)):
                denominator += alpha[t, i] * A[j, i] * B[j, int(O[t+1])] * beta[t+1, j]
        for i in range(len(A)):
            for j in range(len(A)):
#                 print(B.shape, j, O[t+1])
                numerator = alpha[t, i] * A[j, i] * B[j, int(O[t+1])] * beta[t + 1, j]
                zeta[t, i, j] = numerator / denominator
    
    return gamma, zeta
    

In [ ]:
def MStep(O, A, B, gamma, zeta, c, alpha, beta):
    for i in range(len(A)):
        for j in range(len(A)):
            numerator = 0
            denominator = np.sum(zeta[:-1, i, :])
            numerator = np.sum(zeta[:, i, j])
            A[j, i] = numerator / denominator

        for o in range(len(B[0])):
            numerator = 0
            denominator = 0
            for t in range(len(O)):
                if O[t] == o:
                    numerator += gamma[t][i]
            B[i][o] = numerator / np.sum(gamma[:, i])
    
    return A, B

In [ ]:
def baum_welch(O, A, B, pi, lMax):
    for l in range(lMax):
        print("Loop " + str(l + 1) + " / " + str(lMax))
        alpha, c = forward(O, A, B, pi)
#         print("Alpha done")
        beta = backward(O, A, B, c)
        for i in range(len(B)):
            for j in range(len(B[0])):
                if B[i, j] == 0:
                    B[i, j] = 1e-8
#         print("Beta done")
        
        gamma, zeta = EStep(O, A, B, alpha.copy(), beta.copy(), c)
#         print("EStep done")
        A, B = MStep(O, A, B, gamma, zeta, c, alpha, beta)
#         print("MStep done")
        for i in range(len(B)):
            for j in range(len(B[0])):
                if B[i, j] == 0:
                    B[i, j] = 1e-8
#     Check alphas
#     for t in range(len(O)):
#         print(alpha[t] / np.prod(c[:t+1]))
#     Check betas
#     for t in range(len(O)):
#         print(beta[t] / np.prod(c[t:]))
    
    return A, B

In [ ]:
def buildHMM(gestureType, data_dir, K, M, pi):
    print("Building HMM for " + gestureType + "...")
    kmeans = trainKMeans(gestureType, data_dir, M)
    data = np.atleast_2d(kmeans.labels_.astype(int)).T
    
    # Generate A as left to right transition matrix
    A = np.zeros([K, K])
    A[0, 0] = 0.5
    A[0, -1] = 0.5
    for i in range(K - 1):
        A[i + 1, i] = 0.5
        A[i + 1, i + 1] = 0.5
    
    # Generate B as randoms
    B = np.random.rand(K, M)
    for row in range(len(B)):
    #     print(np.sum(B[row]))
        B[row] = B[row] / np.sum(B[row])
    A, B = baum_welch(data.copy(), A.copy(), B.copy(), pi.copy(), 10)
    print("Done!")

    return A, B, kmeans
    

# Test Trained Data

In [ ]:
def testKMeans(data_dir, filename, kmeans):
    data = np.array([])
    print("Loading training data...")
    temp = np.loadtxt(data_dir + filename)
    print(temp.shape)
    if len(data) == 0:
        data = temp[:,1:]
    else:
        data = np.concatenate((data, temp[:,1:]))

    kmeansTest = kmeans.predict(data)
    print("Done!")
    
#     plt.figure(figsize=(16, 12))
#     for i in range(data.shape[1] - 1):
#         plt.scatter(data[:,i], data[:,i+1], c=kmeansTest, s=0.4)
#     for i in range(data.shape[1] - 1):
#         plt.scatter(kmeans.cluster_centers_[:,i], kmeans.cluster_centers_[:,i+1], c='r', s=10)
#     # plt.scatter(data[:,0], data[:,1])
    data = np.atleast_2d(kmeansTest).T
    return data

In [ ]:
def score(data, A, B, pi):
    alpha, c = forward(data, A, B, pi)
    p = 0
    for ct in c:
        p += math.log(ct)
    return -p

# Test With Toy Data

In [ ]:
X=np.concatenate((np.zeros((1000,1)),np.random.randint(0,2,size=(1000,1)),np.zeros((1000,1))))
# Y = np.array([[0, 1]*50]).T
# X=np.concatenate((np.zeros((100,1)),Y,np.zeros((100,1))))
# print(Y)
X.shape

In [ ]:
K = 2
M = 2
A = np.ones([K, K])
# A = np.array([[.25, .75], [0.5, 0.5]])
A = A / len(A)

# B = np.ones([K, M])
# B = B / len(B[0])
B = np.array([[.25, .75], [0.5, 0.5]])
pi = np.zeros(K)
pi[0] = 1
A, B = baum_welch(X.copy(), A.copy(), B.copy(), pi.copy(), 20)

A.T, B

# Build HMM for Each Gesture

In [ ]:
K = 10
M = 50
pi = np.zeros(K)
pi[0] = 1
# pi = np.ones(K)
# pi /= K
data_dir = "ECE5242Proj2-train/"
waveA, waveB, kmeansWave = buildHMM("wave", data_dir, K, M, pi)
circleA, circleB, kmeansCircle = buildHMM("circle", data_dir, K, M, pi)
eightA, eightB, kmeansEight = buildHMM("eight", data_dir, K, M, pi)
infA, infB, kmeansInf = buildHMM("inf", data_dir, K, M, pi)
beat3A, beat3B, kmeansBeat3 = buildHMM("beat3", data_dir, K, M, pi)
beat4A, beat4B, kmeansBeat4 = buildHMM("beat4", data_dir, K, M, pi)

# Evaluate Unknown Gesture

In [ ]:
data_dir = "ECE5242Proj2_train_additional/"
filename = "wave31.txt"
# Get test clusters
# testData = testKMeans(data_dir, filename, kmeansEight)
key = ["Wave", "Circle", "Eight", "Inf", "Beat3", "Beat4"]

# Test for wave
testData = testKMeans(data_dir, filename, kmeansWave)
waveScore = score(testData, waveA, waveB, pi)
# Test for circle
testData = testKMeans(data_dir, filename, kmeansCircle)
circleScore = score(testData, circleA, circleB, pi)
# Test for eight
testData = testKMeans(data_dir, filename, kmeansEight)
eightScore = score(testData, eightA, eightB, pi)
# Test for inf
testData = testKMeans(data_dir, filename, kmeansInf)
infScore = score(testData, infA, infB, pi)
# Test for beat3
testData = testKMeans(data_dir, filename, kmeansBeat3)
beat3Score = score(testData, beat3A, beat3B, pi)
# Test for beat4
testData = testKMeans(data_dir, filename, kmeansBeat4)
beat4Score = score(testData, beat4A, beat4B, pi)

scores = [waveScore, circleScore, eightScore, infScore, beat3Score, beat4Score]
for idx in range(len(scores)):
    if np.isnan(scores[idx]):
        scores[idx] = -float('inf')
        
pairs = sorted(zip(scores, key), reverse=True)
for pair in pairs:
    print(pair[1] + " Score: " + str(pair[0]))

maxIdx = scores.index(max(scores))
print("Detected Gesture: " + pairs[0][1])

In [ ]:
# wave50 = (waveA, waveB, kmeansWave)
# circle50 = (circleA, circleB, kmeansCircle)
# eight50 = (eightA, eightB, kmeansEight)
# inf50 = (infA, infB, kmeansInf)
# beat350 = (beat3A, beat3B, kmeansBeat3)
# beat450 = (beat4A, beat4B, kmeansBeat4)